# Simulated beam pattern

## Pre-processing beam pattern data and getting GSM maps.

In [1]:
import numpy as np
import pandas as pd
import os
import glob
from pygsm import GlobalSkyModel

In [2]:
def clean_folder():
    '''
    DESCRIPTION: Clean the folder "antenna_beam" before processing and separating 
    the beam pattern of some antenna (HIBiscus or Mango Peel)
    '''
    
    files = glob.glob('antenna_beam/*')
    for f in files:
        os.remove(f)

def hibiscus(pathtbeam = 'HIBiscus/'):
    '''
    DESCRIPTION: Process the beam pattern of the HIBiscus antenna and puts the 
    separated hdf5 files in the "antenna_beam" folder. Each frequency
    has an individual file e.g. "050MHz.hdf5"
    
    INPUT: pathbeam = path of folder with beam pattern .txt files
    '''
    
    clean_folder()
    
    Freqs = np.arange(50,91)
    for f in Freqs:
        infile = pathtbeam+"0%dMHz.txt"%f
        Data = np.loadtxt(infile)
        df = pd.DataFrame(data = Data,columns = ['Theta','Phi','dB']) 
        df.to_hdf('antenna_beam/0%dMHz.hdf5'%f,'df')
        
def mango_peel(file = "mango_peel_beam_pattern.dat"):
    '''
    DESCRIPTION: Process the beam pattern of the Mango Peel or the HIBiscus antenna and puts the 
    separated hdf5 files in the "antenna_beam" folder. Each frequency
    has an individual file e.g. "050MHz.hdf5"
    
    INPUT: file = location of beam pattern .dat file
    '''
    
    clean_folder()
    
    df = pd.read_csv(file, sep=", ", 
                 index_col=0, header=None, engine = "python") 
    
    new_header = df.iloc[0] 
    df = df[1:]
    df.columns = new_header
    df.reset_index(inplace=True)
    df.rename(columns = {0:"THETA"}, inplace = True)
    
    theta = list(df["THETA"])
    phi = list(df["'PHI'"])
    
    Freq = np.arange(30000000.0, 150000000.0, 1000000)

    for f in Freq:
        data = df[f]

        file = pd.DataFrame(list(zip(theta, phi, data)), columns =['Theta', 'Phi', 'dB'])

        file["Theta"] = pd.to_numeric(file["Theta"], downcast="float")
        file["Phi"] = pd.to_numeric(file["Phi"], downcast="float")
        file["dB"] = pd.to_numeric(file["dB"], downcast="float")

        name = "0" + str(int(f/(1e+6))) + "MHz"
        file.to_hdf('antenna_beam/'+name+'.hdf5','df')

In [8]:
if not os.path.exists('antenna_beam'):
     os.makedirs('antenna_beam')

#Remove the comment as needed (depending on which antenna you´re working on)

##mango_peel()
#hibiscus()

In [9]:
# This code loads the GSM maps files in the folder "gsm_maps"

if not os.path.exists('gsm_maps'):
    os.makedirs('gsm_maps')
        
    Freqs = np.arange(50,91) #Frequency of interest 50-90 MHz.
    for f in Freqs:
        gsm = GlobalSkyModel()
        gsm_map = gsm.generate(f)
        DATA = pd.DataFrame(gsm_map)
        DATA.to_hdf('gsm_maps/gsm_%dMHz.hdf5'%f,'df')